In [0]:
dbutils.widgets.text("file_name","")

p_file_name=dbutils.widgets.get("file_name")

In [0]:

df = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option("cloudFiles.schemaLocation",f"abfss://bronze@advenworksdl.dfs.core.windows.net/checkpoint_{p_file_name}") \
    .load(f"abfss://source@advenworksdl.dfs.core.windows.net/{p_file_name}")

In [0]:
query=df.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation",f"abfss://bronze@advenworksdl.dfs.core.windows.net/checkpoint_{p_file_name}")\
    .option("path",f"abfss://bronze@advenworksdl.dfs.core.windows.net/{p_file_name}")\
    .trigger(once=True)\
    .start()

query.awaitTermination() 

query = f"""
CREATE TABLE IF NOT EXISTS advenworks_cata.bronze.{p_file_name}
USING DELTA
LOCATION 'abfss://bronze@advenworksdl.dfs.core.windows.net/{p_file_name}'
"""
spark.sql(query)



DataFrame[]